## Installing Required Libraries

In [ ]:
!pip install transformers -U -q

     |████████████████████████████████| 4.0 MB 8.8 MB/s 
     |████████████████████████████████| 596 kB 30.7 MB/s 
     |████████████████████████████████| 6.6 MB 34.5 MB/s 
     |████████████████████████████████| 77 kB 5.0 MB/s 
     |████████████████████████████████| 880 kB 53.7 MB/s 


In [ ]:
! pip install sentencepiece

     |████████████████████████████████| 1.2 MB 7.6 MB/s 


In [ ]:
!pip freeze | grep transformers

transformers==4.18.0


In [ ]:
!pip install datasets
!pip install -U nltk

     |████████████████████████████████| 325 kB 8.0 MB/s 
     |████████████████████████████████| 1.1 MB 60.5 MB/s 
     |████████████████████████████████| 212 kB 59.1 MB/s 
     |████████████████████████████████| 136 kB 49.5 MB/s 
     |████████████████████████████████| 127 kB 62.3 MB/s 
     |████████████████████████████████| 271 kB 36.7 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 144 kB 54.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 1.5 MB 6.2 MB/s 
     |████████████████████████████████| 749 kB 24.1 MB/s

#Importing Libraries

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import os, shutil
import  nltk.translate.bleu_score as bleu
import warnings
warnings.filterwarnings('ignore')

from nltk.translate.meteor_score import meteor_score
from nltk import word_tokenize
import itertools

## Google Authentication

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Accessing the Dataset

In [ ]:
path = '/content/drive/MyDrive/My Documents/NLU/Project/dataset_for_nlu_project'
path_en_bn ='/content/drive/MyDrive/My Documents/NLU/Project/dataset_for_nlu_project/en-bn'
path_en_hi = '/content/drive/MyDrive/My Documents/NLU/Project/dataset_for_nlu_project/en-hi'

Total count of sentences in the dataset for English and corresponding Hindi Translations are 8466307 each. It was taking a long time to train the model on such a large dataset, every time the session was crashing while computation. So for our project we have used the first 200000 data points for traing the model. 

#Data preprocessing  (Run only once)

Stored the first 200000 data of english and corresponding hindi label in a dataframe and exported it to a csv file, which will later be used for train test split (One time task, no need to run this code again)

En-Hi Data

In [ ]:
#@title English-Hindi Data
file = open("/content/drive/MyDrive/My Documents/NLU/Project/dataset_for_nlu_project/en-hi/train.en",'r')
count = 0
en_text = []
line = file.readline() 
while line:
  line = line.rstrip() 
  en_text.append(line)
  line = file.readline()
  count+=1
  if(count>=200000):
    break

file = open("/content/drive/MyDrive/My Documents/NLU/Project/dataset_for_nlu_project/en-hi/train.hi",'r')
hi_text = []
count = 0
line = file.readline()
while line:
  line = line.rstrip() 
  hi_text.append(line)
  line = file.readline()
  count+=1
  if(count>=200000):
    break


In [ ]:
print(len(en_text))
print(len(hi_text))

200000
200000


In [ ]:
print(en_text[1])
print(hi_text[1])

The European Union has seven principal decision-making bodies, its institutions: the European Parliament, the European Council, the Council of the European Union, the European Commission, the Court of Justice of the European Union, the European Central Bank and the European Court of Auditors.
यूरोपीय संघ के महत्वपूर्ण संस्थानों में यूरोपियन कमीशन, यूरोपीय संसद, यूरोपीय संघ परिषद, यूरोपीय न्यायलय एवं यूरोपियन सेंट्रल बैंक इत्यादि शामिल हैं।


In [ ]:
import pandas as pd
df = pd.DataFrame(list(zip(en_text, hi_text)),
               columns =['english', 'hindi'])

In [ ]:
df.to_csv("nlu_dataset.csv")

# Reading the dataset

In [ ]:
dataset_path ="/content/drive/MyDrive/My Documents/NLU/Project/nlu_dataset.csv"
data = pd.read_csv(dataset_path)
del data["Unnamed: 0"]

In [ ]:
X = list(data["english"])
Y = list(data["hindi"])

Train Test Split (80:20)

In [ ]:
trainData,testData,trainLabel,testLabel = train_test_split(X,Y,test_size=0.2,random_state=0)

In [ ]:
print(len(trainData),":",len(trainLabel))
print(len(testData),":",len(testLabel))

160000 : 160000
40000 : 40000


Moving data and labels to GPU if available

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


MBart Pretrained Transformer Model

In [ ]:
model_pretrained = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
#model_pretrained = model_pretrained.to(device)

Downloading:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

In [ ]:
#print(model_pretrained)
type(model_pretrained)

transformers.models.mbart.modeling_mbart.MBartForConditionalGeneration

In [ ]:
print(model_pretrained.model.encoder)

In [ ]:
#Encoder
print(model_pretrained.model.encoder)

MBartEncoder(
  (embed_tokens): Embedding(250054, 1024, padding_idx=1)
  (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
  (layers): ModuleList(
    (0): MBartEncoderLayer(
      (self_attn): MBartAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (activation_fn): ReLU()
      (fc1): Linear(in_features=1024, out_features=4096, bias=True)
      (fc2): Linear(in_features=4096, out_features=1024, bias=True)
      (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    )
    (1): MBartEncoderLayer(
      (self_attn): MBartAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
 

In [ ]:
#Decoder
print(model_pretrained.model.decoder)

MBartDecoder(
  (embed_tokens): Embedding(250054, 1024, padding_idx=1)
  (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
  (layers): ModuleList(
    (0): MBartDecoderLayer(
      (self_attn): MBartAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (activation_fn): ReLU()
      (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder_attn): MBartAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (encoder_attn_

In [ ]:
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")

Downloading:   0%|          | 0.00/528 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/717 [00:00<?, ?B/s]

In [ ]:
article_en = en_text[2]
article_en

'The Congress leader represents Sivaganga Lok Sabha segment from Tamil Nadu.'

In [ ]:
model_inputs = tokenizer(article_en, return_tensors="pt")
#model_inputs =model_inputs.to(device)

In [ ]:
# translate from English to Hindi
generated_tokens = model_pretrained.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"]
)


In [ ]:
translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [ ]:
print("Translated:",translation)
print("Actual:",hi_text[2])

Translated: ['कांग्रेस नेता तमिलनाडु से शिवगंगा लोक सभा का प्रतिनिधित्व करते हैं।']
Actual: कांग्रेस नेता तमिलनाडु से शिवगंगा लोकसभा क्षेत्र का प्रतिनिधित्व करते हैं.


# Training the model on our dataset

In [ ]:
model1 = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
tokenizer1 = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX", tgt_lang="hi_IN")

Trained the model on first 200000 data points and stored the pretrained model for Inferencing.

In [ ]:
#for i in range(len(trainData)):
file_english = "/content/drive/MyDrive/My Documents/NLU/Project/dataset_for_nlu_project/en-hi/train.en"
file_bengali = "/content/drive/MyDrive/My Documents/NLU/Project/dataset_for_nlu_project/en-hi/train.hi"

count=0
with open(file_english,"r") as f1, open(file_bengali,"r") as f2:
  for x, y in zip(f1, f2):
     english_text = x.rstrip()
     #print(english_text)
     hindi_text = y.rstrip()
     #print(hindi_text)
     count=count+1
     model_inputs1 = tokenizer1(english_text, return_tensors="pt") #source text
     #model_inputs1 = model_inputs1.to(device)
     with tokenizer1.as_target_tokenizer():
        labels1 = tokenizer1(hindi_text, return_tensors="pt").input_ids
        #labels1 = labels1.to(device)
     model1(**model_inputs1, labels=labels1)  # forward pass
     if count==200000:
        print("Training done")
        print(model1(**model_inputs1, labels=labels1))
        model1.save_pretrained('/content/drive/MyDrive/My Documents/NLU/Project/model')
        break

Training done
Seq2SeqLMOutput(loss=tensor(2.5830, grad_fn=<NllLossBackward0>), logits=tensor([[[ -4.6119,  -4.7208,  -1.5650,  ...,  -1.5015,   0.1512,  -4.7445],
         [ -0.4310,  -0.6983,   3.5653,  ...,  -3.4063,  -6.2601,  -0.6753],
         [ -0.4609,  -0.5986,   2.5359,  ...,  -6.7224,  -3.8602,  -0.5375],
         ...,
         [ -0.6985,  -0.6922,   5.7594,  ..., -11.1668,  -5.6468,  -0.6232],
         [ -0.2052,  -0.1881,   9.5906,  ...,  -6.3953,  -3.1415,  -0.1763],
         [  0.0754,   0.1942,  14.8099,  ...,  -1.4596,  -1.2649,   0.2265]]],
       grad_fn=<AddBackward0>), past_key_values=None, decoder_hidden_states=None, decoder_attentions=None, cross_attentions=None, encoder_last_hidden_state=tensor([[[ 9.3228e-03,  2.9559e-03, -3.9491e-03,  ..., -1.5571e-03,
           1.8298e-03, -5.0697e-03],
         [ 1.2484e-01,  2.7186e-01,  1.7740e-01,  ..., -2.4486e-01,
           3.1062e-02, -2.8129e-01],
         [ 8.8983e-03, -3.7159e-01,  9.7730e-02,  ..., -9.4952e-01,
  

#Testing

Accessed the trained model and used that for the langugae translation. Evaluation metric which is used for checking the performance is BLUE Score.

In [ ]:
path = "/content/drive/MyDrive/My Documents/NLU/Project/model"
model_trained= MBartForConditionalGeneration.from_pretrained(path)

In [ ]:
model_trained.config

MBartConfig {
  "_name_or_path": "/content/drive/MyDrive/My Documents/NLU/Project/model",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
 

In [ ]:
#model_trained.get_encoder

In [ ]:
#model_trained.get_decoder

#BLEU (Bilingual Evaluation Understudy Score) Score

In [ ]:
def BLEU_calculation(test,predicted):
  reference_translation=test.split()
  candidate_translation_1=predicted.split()
  print("BLEU Score: ",bleu.sentence_bleu(reference_translation, candidate_translation_1))

#ROUGE (Recall Oriented Understudy for Gisting Evaluation) Score

In [ ]:
#Split Sentence into Words
def sentence_split_into_words(sent):
  return list(itertools.chain(*[_.split(" ") for _ in sent]))

#Used to  n-grams for multiple sentences
def get_word_ngrams(n, sent_s):

  assert len(sent_s) > 0
  assert n > 0
  w = sentence_split_into_words(sent_s)
  return get_ngrams(n, w)


#Calcualtes n-grams where n: which n-grams to calculate
def get_ngrams(n, text):
  ngram_set = set()
  text_len = len(text)
  max_index_ngram_start = text_len - n
  for k in range(max_index_ngram_start + 1):
    ngram_set.add(tuple(text[k:k + n]))
  return ngram_set

#Calculate ROUGE Score of Refrence and Evaluated Sentences,  n: Size of ngram
def rouge_n(reference_sentences, evaluated_sentences, n=2):
  if len(evaluated_sentences) <= 0 or len(reference_sentences) <= 0:
    raise ValueError("No word in the sentence")

  eval_ngrams = get_word_ngrams(n, evaluated_sentences)
  ref_ngrams = get_word_ngrams(n, reference_sentences)
  reference_count = len(ref_ngrams)
  eval_count = len(eval_ngrams)

  # Gets the overlapping ngrams between evaluated and reference
  overlap_count = len(eval_ngrams.intersection(ref_ngrams))

  if eval_count == 0:
    precision = 0.0
  else:
    precision = overlap_count / eval_count

  if reference_count == 0:
    recall = 0.0
  else:
    recall = overlap_count / reference_count

  f1_score = 2.0 * ((precision * recall) / (precision + recall + 1e-8))

  return recall


#First testing for 10 test data

In [ ]:
testData1 = testData[:10]
predicted_trans = []
blue_score =[]
rogue_score = []
#meteor_score = []
m_score =[]
for i in range(len(testData1)):
  text = testData1[i]
  model2_inputs = tokenizer(text, return_tensors="pt")
  #model2_inputs = model2_inputs.to(device)
  generated_tokens2 = model_trained.generate(**model2_inputs,forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"])
  translation = tokenizer.batch_decode(generated_tokens2, skip_special_tokens=True)
  reference_translation=testLabel[i]
  candidate_translation_1=translation[0]

  #METEOR Score
  ms = round(meteor_score([word_tokenize(reference_translation)],word_tokenize(candidate_translation_1)), 3)
  m_score.append(ms)

 #BLEU Score
  bs = bleu.sentence_bleu(reference_translation, candidate_translation_1)
  blue_score.append(bs)

 #ROUGE Score
  rs = rouge_n(reference_translation,candidate_translation_1)
  rogue_score.append(rs)

  predicted_trans.append(translation[0])

  if (i% 2 == 0):
    print(testData[i])
    print("Hindi Predicted Translation: ", translation[0])
    print("Ground Truth: ", testLabel[i])
    print("BLEU Score: ",bs)
    print("ROUGE Score:",rs)
    print("METEOR Score:",ms)
    print()
    print("*************************************************************************")
    print()

This reflects the fact that in many programming languages these are the characters that may be used in identifiers.
Hindi Predicted Translation:  यह इस तथ्य को प्रतिबिंबित करता है कि कई प्रोग्रामिंग भाषाओं में ये वे अक्षर हैं जिन्हें पहचानकर्ताओं में प्रयोग किया जा सकता है।
Ground Truth:  यह इस तथ्य को उद्भासित करता है कि कई प्रोग्रामिंग भाषाओँ में ये ही वे अक्षर हैं जिनका प्रयोग पहचान चिह्न के रूप में किया जा सकता है।
BLEU Score:  0.6899172552679737
ROUGE Score: 0.7777777777777778
METEOR Score: 0.709

*************************************************************************

Lance nematode parasitize many types of plants.
Hindi Predicted Translation:  लांस सूत्रकृमि अनेक प्रकार के पौधों को परजीवी बनाती है.
Ground Truth:  बल्लमनुमा सूत्रकृमी अनेक प्रकार के पादपों में पराश्रयी होते हैं
BLEU Score:  0.7860753021519787
ROUGE Score: 0.5185185185185185
METEOR Score: 0.292

*************************************************************************

All devotees of the mutt have backed Sharief

#Testing the data for first 200 test data and calculating different Evaluation Scores

In [ ]:
testData = testData[:200]
predicted_trans = []
blue_score =[]
rogue_score = []
#meteor_score = []
m_score =[]
for i in range(len(testData)):
  text = testData[i]
  model2_inputs = tokenizer(text, return_tensors="pt")
  #model2_inputs = model2_inputs.to(device)
  generated_tokens2 = model_trained.generate(**model2_inputs,forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"])
  translation = tokenizer.batch_decode(generated_tokens2, skip_special_tokens=True)
  reference_translation=testLabel[i]
  candidate_translation_1=translation[0]

  #METEOR Score
  ms = round(meteor_score([word_tokenize(reference_translation)],word_tokenize(candidate_translation_1)), 3)
  m_score.append(ms)

 #BLEU Score
  bs = bleu.sentence_bleu(reference_translation, candidate_translation_1)
  blue_score.append(bs)

 #ROUGE Score
  rs = rouge_n(reference_translation,candidate_translation_1)
  rogue_score.append(rs)

  predicted_trans.append(translation[0])

  if (i% 20 == 0):
    print(testData[i])
    print("Hindi Predicted Translation: ", translation[0])
    print("Ground Truth: ", testLabel[i])
    print("BLEU Score: ",bs)
    print("ROUGE Score:",rs)
    print("METEOR Score:",ms)
    print()
    print("*************************************************************************")
    print()

This reflects the fact that in many programming languages these are the characters that may be used in identifiers.
Hindi Predicted Translation:  यह इस तथ्य को प्रतिबिंबित करता है कि कई प्रोग्रामिंग भाषाओं में ये वे अक्षर हैं जिन्हें पहचानकर्ताओं में प्रयोग किया जा सकता है।
Ground Truth:  यह इस तथ्य को उद्भासित करता है कि कई प्रोग्रामिंग भाषाओँ में ये ही वे अक्षर हैं जिनका प्रयोग पहचान चिह्न के रूप में किया जा सकता है।
BLEU Score:  0.6899172552679737
ROUGE Score: 0.7777777777777778
METEOR Score: 0.709

*************************************************************************

Copyright 2006 - 2012 Red Hat, Inc.
Hindi Predicted Translation:  कॉपीराइट 2006-2012 रेड हैट, इंक
Ground Truth:  कॉपीराइट © 2006-2012 Red Hat, Inc.
BLEU Score:  0.8340313804889192
ROUGE Score: 0.5757575757575758
METEOR Score: 0.192

*************************************************************************

As a follow up of the decision taken in April, 2018 by the Prime Ministers of India and Nepal to introduce in

In [ ]:
print("Average Blue Score of 200 test data: ",sum(blue_score)/len(blue_score))
print("Average ROGUE Score of 200 test data: ",sum(rogue_score)/len(rogue_score))
print("Average METEOR Score of 200 test data: ",sum(m_score)/len(m_score))

Average Blue Score of 200 test data:  0.7486066596316231
Average ROGUE Score of 200 test data:  0.6139830834830473
Average METEOR Score of 200 test data:  0.3700199999999999


In [ ]:
df_new = pd.DataFrame(list(zip(testData, testLabel,predicted_trans)),
               columns =['english', 'hindi_ground_truth','hindi_translated'])

In [ ]:
df_new.to_csv("Language_Translation.csv")

In [ ]:
df_new.tail(10)

,english,hindi_ground_truth,hindi_translated
190,"Most of the banks, including State Bank of Ind...",इस हड़ताल को लेकर भारतीय स्टेट बैंक कैनरा बैंक...,"स्टेट बैंक ऑफ इंडिया, पंजाब नेशनल बैंक और बैंक..."
191,This is the thing that we should learn from Au...,यह ऑस्ट्रेलिया से सीखने वाला विषय है।,यही बात हमें ऑस्ट्रेलिया से सीखनी चाहिए।
192,Pakistan bans Hafiz Saeed-led JuD and its char...,"चौतरफा दबाव से फिर झुका पाकिस्तान, हाफिज सईद क...",पाकिस्तान ने हफीज सईद के नेतृत्व में जुडी और उ...
193,Both the Congress and the Aam Aadmi Party (AAP...,कांग्रेस और AAP दोनों ने चुनाव आयोग से मामले क...,कांग्रेस और अम् अम्मी पार्टी (एएएपी) दोनों ही ...
194,But no action was taken on this.,लेकिन इस केस में कोई कार्रवाई नहीं की गई.,लेकिन इस बारे में कोई कार्रवाई नहीं की गई।
195,Computers and telecommunications have transfor...,कम्प्यूटर एवं इंटरनेट ने पूरी दुनिया की तस्वीर...,कंप्यूटर और दूरसंचार ने विश्व को बदल दिया है।
196,"That is Allah, your Lord, there is no god but ...",(लोगों) वही अल्लाह तुम्हारा परवरदिगार है उसके ...,वही अल्लाह तुम्हारा रब है। उसके सिवा कोई पूज्य...
197,The complaint against Dravid by Madhya Pradesh...,द्रविड़ के खिलाफ मध्य प्रदेश क्रिकेट एसोसिएशन ...,मध्य प्रदेश क्रिकेट एसोसिएशन के जीवन सदस्य संज...
198,The area falls under the Civil Lines police st...,मामला सिविल लाइन थाना क्षेत्र का है.,यह क्षेत्र सिविल लाइन पुलिस स्टेशन के अंतर्गत ...
199,Chairmen of Rajya Sabha and Russias Duma call ...,राज्‍य सभा के सभापति और रूस के ‘ड्यूमा’ के चेय...,राज्य सभा और रूसी ड्यूमा के अध्यक्षों से एकजुट...


#Testing on user input

In [ ]:
path = "/content/drive/MyDrive/My Documents/NLU/Project/model"
model_trained= MBartForConditionalGeneration.from_pretrained(path)
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")

def english_hindi_translator(eng_text):
  mod_inputs = tokenizer(eng_text, return_tensors="pt")
  generated_tokens2 = model_trained.generate(**mod_inputs,forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"])
  hindi_translation = tokenizer.batch_decode(generated_tokens2, skip_special_tokens=True)
  return hindi_translation[0]


In [ ]:
eng_t = input("Enter your English text: ")
hindi_translation = english_hindi_translator(str(eng_t))
print("Hindi Translation: ",hindi_translation)

Enter your English text: The Khejarli Massacre was an inspiration for the 20th century environmentalist Chipko movement. Several temples and a cenotaph in Khejarli commemorate the massacre, and the village is the site of an annual Bishnoi ceremony held in honor of the event. A fair is organized every year on Shukla Dashmi of Bhadrapad (Hindi month) in September at Khejarli, Jodhpur, to pay homage to the great martyrs of environment protection.
Hindi Translation:  खेजरली नरसंहार 20वीं शताब्दी के पर्यावरणवादी चिपको आंदोलन के लिए प्रेरणादायक था। खेजरली में कई मंदिर और एक स्मारक इस नरसंहार को याद करते हैं और गांव में इस समारोह के सम्मान में आयोजित एक वार्षिक बीशनोई समारोह का स्थल है। हर वर्ष सितंबर में खेजरली, जोधपुर में पर्यावरण संरक्षण के महान शहीदों को श्रद्धांजलि अर्पित करने के लिए भाद्रापाद के शुक्ल दासमी पर एक मेला आयोजित किया जाता है।
